In [1]:
using CSV, DataFrames, Statistics, Random, Plots, JuMP, Gurobi, Distributions, Random
using StatsBase: sample

#import sklearn functions 
using ScikitLearn
@sk_import linear_model: LogisticRegression
@sk_import linear_model: LinearRegression
@sk_import metrics:accuracy_score;
@sk_import metrics:precision_score;
@sk_import metrics:recall_score;
@sk_import metrics:mean_squared_error;

#import own functions
include("utils.jl")

┌ Info: mkl not found, proceeding to installing non-mkl versions of sci-kit learn via Conda
└ @ ScikitLearn.Skcore /Users/norahallqvist/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:191
┌ Info: Running `conda install -y -c conda-forge 'scikit-learn>=1.2,<1.3'` in root environment
└ @ Conda /Users/norahallqvist/.julia/packages/Conda/2lg2O/src/Conda.jl:127


done
Solving environment: ...working... 

done



# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0




check_accuracy (generic function with 1 method)

## Helper functions

In [2]:
function generate_covariate_shift(X_test)
    #set a seed for distribution
    Random.seed!(123)

    col_means = mean(Matrix(X_test), dims=1)
    std_dev = 0.1

    random_values = zeros(size(X_test))
    for col_index in 1:size(X_test, 2)
        col_mean = col_means[col_index]
        col_shift = rand(Normal(col_mean * 0.5 , std_dev), (size(X_test, 1), 1))
        random_values[:, col_index] = col_shift
    end

    X_test_shift = Matrix(X_test) .+ random_values;
    df_X_test_shift = DataFrame(X_test_shift,Symbol.(names(X_test)))
    return df_X_test_shift
end

function normalize_data(X_train, X_test)
    # Calculate mean and standard deviation from training data
    mean_vals = mean(X_train, dims=1)
    std_vals = std(X_train, dims=1)

    # Normalize training data
    X_train_norm = (X_train .-mean_vals) ./ std_vals;
    X_test_norm = (X_test .-mean_vals) ./ std_vals;

    return X_train_norm, X_test_norm
end

normalize_data (generic function with 1 method)

In [3]:
function mse(y_true, y_pred)
    return sum((y_true .- y_pred).^2) / length(y_true)
end

mse (generic function with 1 method)

In [4]:
function train_test_split(X_full, X_full_shifted, y_full, train_fraction, random_seed = 123)
    
    Random.seed!(random_seed)

    num_indices = round(Int, train_fraction * length(y_full))
    train_indices = sample(1:length(y_full), num_indices, replace=false)
    test_indices = setdiff(1:length(y_full), train_indices)
    
    train_X, train_y = X_full[train_indices, :], y_full[train_indices]
    test_X, test_y = X_full_shifted[test_indices, :], y_full[test_indices]
    
    return (train_X, train_y), (test_X, test_y)
end

train_test_split (generic function with 2 methods)

In [5]:
function LassoRegression(X, y, lambda, weight = "nothing")
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)
    n, p = size(X)

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, u[1:n])
    @variable(model, beta[1:p])
    @variable(model, w[1:p])

    @objective(model, Min, sum(u) + lambda * sum(w))

    for i in 1:p
        @constraint(model, w[i] >= beta[i])
        @constraint(model, w[i] >= -beta[i])
    end

    if weight !== "nothing"
        for i in 1:n
            @constraint(model, u[i] >= weight[i] * (y[i] - sum(X[i, :] .* beta)))
            @constraint(model, u[i] >= - weight[i] * (y[i] - sum(X[i, :] .* beta)))
        end
    else
        for i in 1:n
            @constraint(model, u[i] >= (y[i] - sum(X[i, :] .* beta)))
            @constraint(model, u[i] >=  - (y[i] - sum(X[i, :] .* beta)))
        end
    end

    optimize!(model)

    return value.(beta)
end


LassoRegression (generic function with 2 methods)

In [6]:
function optimized_split(X, y, lambda, train_fraction, weights = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)

    #parameters
    n, p = size(X)
    k = n * train_fraction

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, theta)
    @variable(model, u[1:n] >= 0)
    @variable(model, beta[1:p])
    @variable(model, w[1:p])

    @objective(model, Min, k * theta + sum(u) + lambda * sum(w))

    for i in 1:p
        @constraint(model, w[i] >= beta[i])
        @constraint(model, w[i] >= -beta[i])
    end 

    for i in 1:n
        if weights == nothing
            @constraint(model, theta + u[i] >= y[i] - sum(X[i, :].*beta))
            @constraint(model, theta + u[i] >= -(y[i] - sum(X[i, :].*beta)))
        else 
            @constraint(model, theta + u[i] >=  weights[i] * (y[i] - sum(X[i, :].*beta)))
            @constraint(model, theta + u[i] >= - weights[i] * (y[i] - sum(X[i, :].*beta)))
        end 
    end
    
    optimize!(model)

    return value(theta), value.(u), value.(beta), value.(w)
end

optimized_split (generic function with 2 methods)

In [7]:
function split_train_val(X, y, beta_opt, train_fraction, weights = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)

    residuals = y - X * beta_opt

    if weights == nothing
        sorted_indices = sortperm(abs.(residuals), rev=true)
    else 
        residuals_weights = [residuals[i] * weights[i] for i in 1:length(weights)]
        sorted_indices = sortperm(abs.(residuals_weights), rev=true)
    end

    num_train_points = round(Int, train_fraction * length(sorted_indices))

    train_indices = sorted_indices[1:num_train_points]

    val_indices = setdiff(1:length(y), train_indices)

    X_train = X[train_indices, :] 
    y_train = y[train_indices]

    X_val = X[val_indices, :]
    y_val = y[val_indices]

    return X_train, y_train, X_val, y_val
end


split_train_val (generic function with 2 methods)

In [8]:
function get_optimized_split_test_score(X_full_train, y_full_train, X_test, y_test, weights, lambdas, train_fraction, print_result=false)
    best_lambda = Inf
    best_val_mse = Inf
    best_model = Inf

    #need to add ones for prediction
    X_test_with_intercept = hcat(ones(Int, size(X_test, 1)), X_test)
    
    for lambda in lambdas
        # Get optimized split
        _, _, betas, _ = optimized_split(X_full_train, y_full_train, lambda, train_fraction, weights)
        #ones have been added in split_train_val function 
        X_train, y_train, X_val, y_val = split_train_val(X_full_train, y_full_train, betas, train_fraction, weights)
        
        # Predict on validation set
        y_pred_val =  X_val * betas
        val_mse_i = mse(y_val, y_pred_val)
        
        if best_val_mse > val_mse_i
            best_lambda = lambda
            best_val_mse = val_mse_i
            best_model = betas
        end
    end

    #get mse on test set for best performing model 
    y_pred_test = X_test_with_intercept * best_model
    mse_test_mse = mse(y_test, y_pred_test)

    if print_result
        println("Best lambda: ", best_lambda)
        println("Validation score: ", best_val_mse)
        println("Test score: ", mse_test_mse)
        println("Number of betas: ", length(best_model))
    end

    return mse_test_mse
end

get_optimized_split_test_score (generic function with 2 methods)

In [9]:
function get_random_split_test_score(X_full_train, y_full_train, X_test, y_test, weights, lambdas, train_fraction, print_result=false)
    best_lambda = Inf
    best_val_mse = Inf
    best_model = Inf

    (X_train, y_train), (X_val, y_val) = 
        IAI.split_data(:regression, X_full_train, y_full_train, train_proportion=train_fraction, seed = 209)

    #need to add ones for prediction
    X_val_with_intercept = hcat(ones(Int, size(X_val, 1)), X_val)
    X_test_with_intercept = hcat(ones(Int, size(X_test, 1)), X_test)

    for lambda in lambdas

        betas = LassoRegression(X_train, y_train, lambda, weights)

        y_pred_val = X_val_with_intercept * betas
        val_mse_i = mse(y_val, y_pred_val)

        if best_val_mse > val_mse_i
            best_lambda = lambda
            best_val_mse = val_mse_i
            best_model = betas
        end
    end

    #get test mse score on best performing model
    y_pred_test = X_test_with_intercept * best_model
    mse_test_mse = mse(y_test,y_pred_test)

    if print_result
        println("Best lambda: ", best_lambda)
        println("Validation score: ", best_val_mse)
        println("Test score: ", mse_test_mse)
        println("Number of betas: ", length(best_model))
    end

    return mse_test_mse
end


get_random_split_test_score (generic function with 2 methods)

In [10]:
function prepare_data(X_full, X_shift_full, y_full, random_seed, train_frac)

    (X_train, y_train), (X_test, y_test) = train_test_split(Matrix(X_full), Matrix(X_shift_full), y_full, train_frac, random_seed);
    X_train_norm, X_test_norm = normalize_data(X_train, X_test);
    
    return (X_train_norm, y_train), (X_test_norm, y_test)
end

prepare_data (generic function with 1 method)

In [11]:
function repeat_stable_reg_with_weights(X_full, X_shift_full, y_full, lambdas, train_fraction = 0.7, num_repeats = 100, optimised_method = true)

    train_test_frac = 0.9
    train_val_frac = 0.7
    test_mse_scores = []

    for random_seed in 1:num_repeats
    
        #split and normalise data
        (X_train_norm, y_train), (X_test_norm, y_test) = prepare_data(X_full, X_shift_full, y_full, random_seed, train_test_frac);
        weights = get_weights(X_train_norm, X_test_norm, false);

        if random_seed % 10 == 0
            println(random_seed)
        end
        
        if optimised_method
            mse_test_score = get_optimized_split_test_score(X_train_norm, y_train, X_test_norm, y_test, weights, lambdas, train_fraction)
        else 
            mse_test_score = get_random_split_test_score(X_train_norm, y_train, X_test_norm, y_test, weights, lambdas, train_fraction)
        end
        push!(test_mse_scores, mse_test_score)
    end 
    
    return test_mse_scores
end


repeat_stable_reg_with_weights (generic function with 4 methods)

## Try with concrete data

In [12]:
random_seed = 123
num_runs = 1
train_fraction = 0.7 
lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1];

X_full, y_full = get_concrete_data()
X_shifted_full = generate_covariate_shift(X_full);

In [46]:
test_mse_scores_opt = repeat_stable_reg_with_weights(X_full, X_shifted_full, y_full, lambdas, train_fraction, num_runs, true);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [47]:
test_mse_scores_opt

1-element Vector{Any}:
 111.31613073748223

In [48]:
test_mse_scores_rand = repeat_stable_reg_with_weights(X_full, X_shifted_full, y_full, lambdas, train_fraction, num_runs, false);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [49]:
test_mse_scores_rand

1-element Vector{Any}:
 129.61338635203984

In [40]:
concrete_improvment = calculate_percentage_improvement(test_mse_scores_opt,test_mse_scores_rand)
println("% improvment for concrete dataset: ",  concrete_improvment)

% improvment for concrete dataset: 37.61312796940281


### Abalone

In [15]:
X_full, y_full = get_abalone_data()
X_shifted_full = generate_covariate_shift(X_full);

In [16]:
test_mse_scores_opt = repeat_stable_reg_with_weights(X_full, X_shifted_full, y_full, lambdas, train_fraction, num_runs, true);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

In [17]:
test_mse_scores_rand = repeat_stable_reg_with_weights(X_full, X_shifted_full, y_full, lambdas, train_fraction, num_runs, false);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

In [18]:
concrete_improvment = calculate_percentage_improvement(test_mse_scores_opt,test_mse_scores_rand)
println("% improvment for concrete dataset: ",  concrete_improvment)

% improvment for concrete dataset: 51.7784491017051


### Computer hardware

In [19]:
X_full, y_full = get_concrete_data()
X_shifted_full = generate_covariate_shift(X_full);

In [ ]:
test_mse_scores_opt = repeat_stable_reg_with_weights(X_full, X_shifted_full, y_full, lambdas, train_fraction, num_runs, true);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

In [ ]:
test_mse_scores_rand = repeat_stable_reg_with_weights(X_full, X_shifted_full, y_full, lambdas, train_fraction, num_runs, false);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

In [ ]:
concrete_improvment = calculate_percentage_improvement(test_mse_scores_opt,test_mse_scores_rand)
println("% improvment for concrete dataset: ",  concrete_improvment)

% improvment for concrete dataset: 33.12862763975939
